In [2]:
import matplotlib.pyplot as plt
import numpy as np
from qctrlvisualizer import get_qctrl_style
from scipy.linalg import expm

from qctrl import Qctrl

plt.style.use(get_qctrl_style())

# Define standard matrices
sigma_x = np.array([[0, 1], [1, 0]], dtype=np.complex)
sigma_y = np.array([[0, -1j], [1j, 0]], dtype=np.complex)
sigma_z = np.array([[1, 0], [0, -1]], dtype=np.complex)

# Start a session with the API
qctrl = Qctrl()

### Ideal Qubit

$$\frac{H(t)}{\hbar} = \frac{1}{2} \Omega(t) b + \frac{1}{2} \Omega^\ast(t) b^\dagger$$


In [2]:
def simulate_ideal_qubit(
    duration=1, values=np.array([np.pi]), shots=1024, repetitions=1
):

    b = np.array([[0, 1], [0, 0]])  # Lowering operator
    initial_state = np.array([[1], [0]])  # Initial state of qubit in |0>

    with qctrl.create_graph() as graph:

        # Create time dependent \Omega(t)
        drive = qctrl.operations.pwc_signal(duration=duration, values=values)

        # Construct Hamiltonian (\Omega(t) b + \Omega^*(t) b^\dagger)/2
        hamiltonian = qctrl.operations.pwc_operator_hermitian_part(
            qctrl.operations.pwc_operator(signal=drive, operator=b)
        )

        # Solve Schrodinger's equation and get total unitary at the end
        unitary = qctrl.operations.time_evolution_operators_pwc(
            hamiltonian=hamiltonian,
            sample_times=np.array([duration]),
        )[-1]
        unitary.name = "unitary"

        # Repeat final unitary
        repeated_unitary = np.eye(2)
        for _ in range(repetitions):
            repeated_unitary = repeated_unitary @ unitary
        repeated_unitary.name = "repeated_unitary"

        # Calculate final state.
        state = repeated_unitary @ initial_state

        # Calculate final populations.
        populations = qctrl.operations.abs(state[:, 0]) ** 2
        # Normalize populations because of numerical precision
        norm = qctrl.operations.sum(populations)
        populations = populations / norm
        populations.name = "populations"

    # Evaluate graph.
    result = qctrl.functions.calculate_graph(
        graph=graph,
        output_node_names=["unitary", "repeated_unitary", "populations"],
    )

    # Extract outputs.
    unitary = result.output["unitary"]["value"]
    repeated_unitary = result.output["repeated_unitary"]["value"]
    populations = result.output["populations"]["value"]

    # Sample projective measurements.
    measurements = np.random.choice(2, size=shots, p=populations)

    results = {"unitary": unitary, "measurements": measurements}

    return results


### Realistic Qubit Hamiltonian

$$\frac{H(t)}{\hbar} = \epsilon_D(t) b^\dagger b + \frac{1}{2} \Omega(t) b + \frac{1}{2} \Omega(t) b^\dagger $$
$$ \frac{H(t)}{\hbar} = \frac{1}{2} (I(t)(1 + \epsilon_I(t)) + Q(t)(1 + \epsilon_Q(t))) b + \mbox{ h. c. } $$

In [3]:
def simulate_more_realistic_qubit(
    duration=1, values=np.array([np.pi]), shots=1024, repetitions=1
):

    # 1. Limits for drive amplitudes
    assert np.amax(values) <= 1.0
    assert np.amin(values) >= -1.0
    max_drive_amplitude = 2 * np.pi * 20  # MHz

    # 2. Dephasing error
    dephasing_error = -2 * 2 * np.pi  # MHz

    # 3. Amplitude error
    amplitude_i_error = 0.98
    amplitude_q_error = 1.03

    # 4. Control line bandwidth limit
    cut_off_frequency = 2 * np.pi * 10  # MHz
    resample_segment_count = 1000

    # 5. SPAM error confusion matrix
    confusion_matrix = np.array([[0.99, 0.01], [0.02, 0.98]])

    # Lowering operator
    b = np.array([[0, 1], [0, 0]])
    # Number operator
    n = np.diag([0, 1])
    # Initial state
    initial_state = np.array([[1], [0]])

    with qctrl.create_graph() as graph:
        # Apply 1. max Rabi rate.
        values = values * max_drive_amplitude

        # Apply 3. amplitude errors.
        values_i = np.real(values) * amplitude_i_error
        values_q = np.imag(values) * amplitude_q_error
        values = values_i + 1j * values_q

        # Apply 4. bandwidth limits
        drive_unfiltered = qctrl.operations.pwc_signal(duration=duration, values=values)
        drive_filtered = qctrl.operations.convolve_pwc(
            pwc=drive_unfiltered,
            kernel_integral=qctrl.operations.sinc_integral_function(cut_off_frequency),
        )
        drive = qctrl.operations.discretize_stf(
            drive_filtered, duration=duration, segments_count=resample_segment_count
        )

        # Construct microwave drive
        drive_term = qctrl.operations.pwc_operator_hermitian_part(
            qctrl.operations.pwc_operator(signal=drive, operator=b)
        )

        # Construct 2. dephasing term.
        dephasing_term = qctrl.operations.constant_pwc_operator(
            operator=dephasing_error * n,
            duration=duration,
        )

        # Construct Hamiltonian.
        hamiltonian = qctrl.operations.pwc_sum(
            [
                drive_term,
                dephasing_term,
            ]
        )

        # Solve Schrodinger's equation and get total unitary at the end
        unitary = qctrl.operations.time_evolution_operators_pwc(
            hamiltonian=hamiltonian,
            sample_times=np.array([duration]),
        )[-1]
        unitary.name = "unitary"

        # Repeat final unitary
        repeated_unitary = np.eye(2)
        for _ in range(repetitions):
            repeated_unitary = repeated_unitary @ unitary
        repeated_unitary.name = "repeated_unitary"

        # Calculate final state.
        state = repeated_unitary @ initial_state

        # Calculate final populations.
        populations = qctrl.operations.abs(state[:, 0]) ** 2
        # Normalize populations
        norm = qctrl.operations.sum(populations)
        populations = populations / norm
        populations.name = "populations"

    # Evaluate graph.
    result = qctrl.functions.calculate_graph(
        graph=graph,
        output_node_names=["unitary", "repeated_unitary", "populations"],
    )

    # Extract outputs.
    unitary = result.output["unitary"]["value"]
    repeated_unitary = result.output["repeated_unitary"]["value"]
    populations = result.output["populations"]["value"]

    # Sample projective measurements.
    true_measurements = np.random.choice(2, size=shots, p=populations)
    measurements = np.array(
        [np.random.choice(2, p=confusion_matrix[m]) for m in true_measurements]
    )

    results = {"unitary": unitary, "measurements": measurements}

    return results


In [14]:
len(results['measurements'])

1024

In [15]:
max_rabi_rate = 20 * 2 * np.pi  # MHz
not_duration = np.pi / (max_rabi_rate)  # us
h_duration = np.pi / (2 * max_rabi_rate)  # us
shots = 1024

values = np.array([1.0])

In [20]:
not_results = simulate_more_realistic_qubit(duration=not_duration, values=values, shots=1024, repetitions=1)

Your task calculate_graph has started.
Your task calculate_graph has completed in 5s.


In [27]:
not_measurements = not_results["measurements"]

def estimate_probability_of_one(measurements):
    size = len(measurements)
    probability = np.mean(measurements)
    standard_error = np.std(measurements) / np.sqrt(size)
    return (probability, standard_error)


not_probability, not_standard_error = estimate_probability_of_one(not_measurements)

In [29]:
measurement_results = not_measurements
measurement_errors = 1 - not_measurements

In [22]:
print("NOT estimated probability of getting 1:" + str(not_probability))
print("NOT estimate standard error:" + str(not_standard_error))

NOT estimated probability of getting 1:0.427734375
NOT estimate standard error:0.01546094119325876


In [46]:
# Define standard matrices
sigma_y = np.array([[0, -1j], [1j, 0]])
sigma_z = np.array([[1, 0], [0, -1]])
sigma_m = np.array([[0, 1], [0, 0]])

# Define physical constants
nu = 2 * np.pi * 0.5 * 1e6  # Hz
gamma_max = 2 * np.pi * 0.5e6  # Hz
alpha_max = 2 * np.pi * 0.5e6  # Hz
segment_count = 50
duration = 10e-6  # s

# Define the data flow graph describing the system
with qctrl.create_graph() as graph:

    nu = qctrl.operations.bounded_optimization_variable(
        count=1, lower_bound=np.pi * 0.5e6, upper_bound=3 * np.pi * 0.5e6, name="nu"
    )
    
    # Create a constant piecewise-constant (PWC) operator representing the
    # detuning term
    detuning = qctrl.operations.constant_pwc_operator(
        duration=duration,
        operator=nu * sigma_z / 2,
    )

    # Create a complex PWC signal, with optimizable modulus and phase,
    # representing gamma(t)
#     gamma = qctrl.operations.complex_pwc_signal(
#         moduli=qctrl.operations.bounded_optimization_variable(
#             count=segment_count,
#             lower_bound=np.pi * 20,
#             upper_bound=3 * np.pi * 20,
#         ),
#         phases=qctrl.operations.unbounded_optimization_variable(
#             count=segment_count,
#             initial_lower_bound=0,
#             initial_upper_bound=2 * np.pi,
#         ),
#         duration=duration,
#         name="gamma",
#     )
#     # Create a PWC operator representing the drive term
#     drive = qctrl.operations.pwc_operator_hermitian_part(
#         qctrl.operations.pwc_operator(signal=gamma, operator=sigma_m)
#     )
    
    drive = qctrl.operations.pwc_signal(duration=duration, values=values)

    # Create a real PWC signal, with optimizable amplitude, representing
    # alpha(t)
    alpha = qctrl.operations.pwc_signal(
        values=qctrl.operations.bounded_optimization_variable(
            count=segment_count, lower_bound=-alpha_max, upper_bound=alpha_max
        ),
        duration=duration,
        name="alpha",
    )
    # Create a PWC operator representing the clock shift term
    shift = qctrl.operations.pwc_operator(signal=alpha, operator=sigma_z / 2)

    # Create a constant PWC operator representing the dephasing noise
    # (note that we scale by 1/duration to ensure consistent units between
    # the noise Hamiltonian and the control Hamiltonian)
    dephasing = qctrl.operations.constant_pwc_operator(
        duration=duration, operator=sigma_z / duration
    )

    # Create the target operator
    target_operator = qctrl.operations.target(operator=sigma_x)

    # Create infidelity
    infidelity = qctrl.operations.infidelity_pwc(
        hamiltonian=qctrl.operations.pwc_sum([detuning, drive, shift]),
        noise_operators=[dephasing],
        target_operator=target_operator,
        name="infidelity",
    )
    
    calculated_points = 1.0 - infidelity
    
    # Calculate cost
    cost = qctrl.operations.sum(
        (calculated_points - measurement_results) ** 2.0
        / (2.0 * measurement_errors ** 2.0),
        name="cost",
    )

    # Calculate Hessian
    hessian = qctrl.operations.hessian_matrix(cost, [nu], name="hessian")

QctrlArgumentsValueError: All the terms must have the same shape.
terms=[TensorPwcNodeData(operation=<qctrlcommons.node.wrapper.Operation object at 0x7feab02b2f10>, values_shape=(2, 2), durations=array([1.e-05]), batch_shape=()), TensorPwcNodeData(operation=<qctrlcommons.node.wrapper.Operation object at 0x7feab02b2a90>, values_shape=(), durations=array([1.e-05]), batch_shape=()), TensorPwcNodeData(operation=<qctrlcommons.node.wrapper.Operation object at 0x7feab0274d50>, values_shape=(2, 2), durations=array([2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07,
       2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07,
       2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07,
       2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07,
       2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07,
       2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07, 2.e-07,
       2.e-07, 2.e-07]), batch_shape=())]

In [45]:
# Estimate the parameters
result = qctrl.functions.calculate_optimization(
    cost_node_name="cost",
    output_node_names=["nu", "hessian"],
    optimization_count=10,
    graph=graph,
)

estimated_nu = result.output["nu"]["value"][0]
estimated_i = result.output["i"]["value"][0]
estimated_q = result.output["q"]["value"][0]

# Calculate 2-sigma uncertainties (error bars give 95% precision)
hessian = result.output["hessian"]["value"]
uncertainties = 2.0 * np.sqrt(np.diag(np.linalg.inv(hessian)))
uncertainty_nu, uncertainty_i, uncertainty_q = uncertainties


RuntimeError: Execution failed: Tensor conversion requested dtype complex128 for Tensor with dtype float64: <tf.Tensor 'strided_slice_1:0' shape=(1, 2, 2) dtype=float64>